In [44]:
import requests
from bs4 import BeautifulSoup
import json
import re
import urllib.parse

In [2]:
class Operator:
    def stations(self):
        pass
    def offers(self,origin:str,destination:str,date:str==None):
        pass

In [79]:
class Renfe (Operator):
    
    def __init__(self,stations_json='renfe.json'):
        stations=json.load(open(stations_json))['stations']
        self.dstations={}
        for e in stations:
            self.dstations[e['station']]=e['id']
    
    def stations(self):
        return self.dstations
    
    def offers(self,origin:str,destination:str,date:str=None):
        if (origin not in self.dstations.keys()):
            raise ValueError('Origin station not found')
        if (destination not in self.dstations.keys()):
            raise ValueError('Destination station not found')
        origin_id=self.dstations[origin]
        destination_id=self.dstations[destination]
        url=f'https://horarios.renfe.com/HIRRenfeWeb/buscar.do?O={origin_id}&D={destination_id}&AF=2022&MF=MM&DF=DD&SF=NaN&ID=s'
        soup=BeautifulSoup(requests.get(url).text, 'html.parser')
        l=soup.find_all('td', {'class':"txt_borde1 irf-travellers-table__td"})
        i=0
        trips=[]
        while i<len(l):
            inf_link='https://horarios.renfe.com/HIRRenfeWeb/'+l[i].a['href'][30:-2].replace('\t','').replace('\n','').replace(' ','%20')
            ls=self.stops(inf_link)
            trips.append(
                {'train':l[i].text.replace('\n', '').replace('\t', '').split(' ')[:2],
                 'stops':ls,
                'timetable':[l[i+1].text.replace('\n', '').replace('\t', ''),l[i+2].text.replace('\n', '').replace('\t', '')],
                'duration':l[i+3].text.replace('\n', '').replace('\t', ''),
                'price':l[i+4].text.replace('\n\n', '').split('\n')[1:]}
            )
            i+=7
        return trips
    
    def stops(self,link):
        soup=BeautifulSoup(requests.get(link).text, 'html.parser')
        l=soup.find_all('td', {'class':"irf-renfe-travel__td txt_gral"})
        i=0
        lStops=[]
        while i<len(l):
            lStops.append(
                {'station':l[i].text.replace('\n', '').replace('\t', '').replace(' ', ''),
                'arrival':l[i+1].text.replace('\n', '').replace('\t', '').replace(' ', ''),
                'departure':l[i+2].text.replace('\n', '').replace('\t', '').replace(' ',''),}
            )
            i+=3
        return lStops

        

In [80]:
r=Renfe('renfe.json')
l=r.offers(origin='Madrid (TODAS)',destination='Barcelona (TODAS)')
AVES=[v for v in l if v['train'][1] in ['AVE']]

In [82]:
for t in AVES:
    print(t['train'],t['stops'])

['03073', 'AVE'] [{'station': 'MADRID-PUERTADEATOCHA', 'arrival': '\xa0', 'departure': '07.30'}, {'station': 'GUADALAJARA-YEBES', 'arrival': '07.53', 'departure': '07.54'}, {'station': 'ZARAGOZA-DELICIAS', 'arrival': '08.51', 'departure': '08.52'}, {'station': 'LLEIDA', 'arrival': '09.35', 'departure': '09.37'}, {'station': 'CAMPTARRAGONA', 'arrival': '10.04', 'departure': '10.06'}, {'station': 'BARCELONA-SANTS', 'arrival': '10.42', 'departure': '10.50'}, {'station': 'GIRONA', 'arrival': '11.28', 'departure': '11.30'}, {'station': 'FIGUERESVILAFANT', 'arrival': '11.45', 'departure': '\xa0'}]
['03093', 'AVE'] [{'station': 'MADRID-PUERTADEATOCHA', 'arrival': '\xa0', 'departure': '09.30'}, {'station': 'ZARAGOZA-DELICIAS', 'arrival': '10.45', 'departure': '10.46'}, {'station': 'LLEIDA', 'arrival': '11.29', 'departure': '11.31'}, {'station': 'CAMPTARRAGONA', 'arrival': '11.58', 'departure': '12.00'}, {'station': 'BARCELONA-SANTS', 'arrival': '12.37', 'departure': '12.45'}, {'station': 'GIRO